In [101]:
import pandas as pd
import numpy as np

In [102]:
df_household = pd.read_excel('household_bmc.xlsx')
df_person = pd.read_excel('person_bmc.xlsx')
#df_vehicle = pd.read_excel('vehicle_bmc.xlsx')
df_trip = pd.read_excel('trip_bmc.xlsx')

df_household.replace(-9, np.nan, inplace=True)
df_person.replace(-9, np.nan, inplace=True)
#df_vehicle.replace(-9, np.nan, inplace=True)
df_trip.replace(-9, np.nan, inplace=True)

print(df_household)
print(df_person)
#print(df_vehicle)
print(df_trip)

      household_id  in_tpb  in_bmc  home_state_fips home_state_puma  \
0        140000006       0       0               24        24_00100   
1        140000013       0       0               24        24_00100   
2        140000017       0       0               24        24_00100   
3        140000019       0       0               24        24_00100   
4        140000024       0       0               24        24_00100   
...            ...     ...     ...              ...             ...   
8308     180256058       1       1               24        24_01201   
8309     180256271       1       1               24        24_00902   
8310     180256360       1       1               24        24_01204   
8311     180256797       1       1               24        24_00902   
8312     180256875       1       1               24        24_01203   

      area_type  home_state_county_fips  home_tract_fips  home_tract_fips20  \
0            10                   24001              100            

In [103]:
def time_difference_minutes(time1, time2):
    hours1 = time1 // 100
    minutes1 = time1 % 100
    hours2 = time2 // 100
    minutes2 = time2 % 100
    total_minutes1 = hours1 * 60 + minutes1
    total_minutes2 = hours2 * 60 + minutes2
    
    return total_minutes2 - total_minutes1

def staying_time(row, df_trip):
    arrival_time = row['arrival_time_hhmm']
    next_ = df_trip.loc[(df_trip['person_id'] == row['person_id']) & (df_trip['tripno'] == row['tripno'] + 1)]
    if not next_.empty:
        next_departure_time = next_['departure_time_hhmm'].iloc[0]
        return time_difference_minutes(arrival_time, next_departure_time)
    else:
        return 0

df_trip['staying_time'] = df_trip.apply(lambda row: staying_time(row, df_trip), axis=1)

In [104]:
#df_consolidated = df_person.copy()
df_consolidated = pd.merge(df_person, df_household, on='household_id', how='outer')
max_trips = df_trip['tripno'].max()
trip_invariants = ['household_id', 'person_id', 'trip_id', 'persno', 'tripno']

# Create an empty list to hold the DataFrames for concatenation
dfs_to_concat = [df_consolidated]
# Iterate over the columns of df_trip
for col in df_trip.columns:
    if col not in trip_invariants:
        # Create a DataFrame with columns named col_i, where i is the trip number
        temp_df = pd.DataFrame(0, index=df_person.index, columns=[f'{col}_{i}' for i in range(1, max_trips + 1)])
        dfs_to_concat.append(temp_df)

# Concatenate all the DataFrames in the list along the axis 1 (columns)
df_consolidated = pd.concat(dfs_to_concat, axis=1)

In [105]:
# Create a dictionary for fast lookup of column indices (excluding 'tripno' and invariant columns)
column_names = [col for col in df_trip.columns if col not in trip_invariants + ['tripno']]

# Loop through each column and populate the updated values in df_consolidated
for col in column_names:
    for tripno in df_trip['tripno'].unique():
        # Construct the new column name
        new_col_name = f'{col}_{tripno}'
        
        # Create a mask for the current tripno
        mask = df_trip['tripno'] == tripno
        
        # Get the values for each 'person_id' and 'tripno' combination
        col_values = df_trip.loc[mask].groupby('person_id')[col].first()

        # Map the values to the 'person_id' in df_consolidated
        df_consolidated[new_col_name] = df_consolidated['person_id'].map(col_values)

# Resulting df_consolidated
print(df_consolidated)

       household_id    person_id  persno transit_commuter  Travel Mode_TAT  \
0         140000006  14000000601       1              NaN              NaN   
1         140000013  14000001301       1              NaN              NaN   
2         140000013  14000001302       2              NaN              NaN   
3         140000017  14000001701       1              NaN              NaN   
4         140000017  14000001702       2              NaN              NaN   
...             ...          ...     ...              ...              ...   
17853     180256058  18025605804       4              NaN              NaN   
17854     180256271  18025627101       1              NaN              NaN   
17855     180256360  18025636001       1              NaN              NaN   
17856     180256797  18025679701       1              NaN              NaN   
17857     180256875  18025687501       1              NaN              NaN   

       Trip Purpose_TAT   age  Non-Motorized Travel  transit_tr

In [106]:
df_consolidated.to_csv('master_trips.csv', index=False)